In [2]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
# import statsmodels.api as sm
import matplotlib
from datetime import datetime


# matplotlib.rcParams['axes.labelsize'] = 14
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
# matplotlib.rcParams['text.color'] = 'k'

In [85]:
## Doing Primary analysis

def getMissingPct(df):
    '''    '''
    return [ round((df['IsNullSum'][i] / max(df['count']))*100,2) for i in range(len(df)) ]

def genMsg(txt):
    '''    '''
    print('_'*12+'| Number of feature/s which are {} : {} |'.format(*txt)+'_'*12)

def DatasetPrimAnalysis(DF):
    '''
    Function which is used to analyze features and provide data insight
    '''
    df_explore = DF.copy()
    print('Overall dataset shape :', df_explore.shape)
    
    ## Creating a dataset that explain feature/s
    temp = pd.DataFrame(df_explore.isnull().sum(), columns = ['IsNullSum'])
    temp['dtypes'] = df_explore.dtypes.tolist()
    temp['IsNaSum'] = df_explore.isna().sum().tolist()
    
    ## Analyzing Time based Features
    temp_tim = temp.loc[temp['dtypes']=='datetime64[ns]' ,:]
    if (len(temp_tim) > 0):
        df_tim = df_explore.loc[:,temp_tim.index].fillna('Missing-NA')
        genMsg(['Time based', df_tim.shape[1]])
        temp_tim = temp_tim.join(df_tim.describe().T).fillna('')
        temp_tim['%Missing'] = getMissingPct(temp_tim)
        display(temp_tim)
    
    
    ## Analyzing Qualitative Features
    temp_cat = temp.loc[temp['dtypes']=='O' ,:]
    if (len(temp_cat) > 0):
        df_cat = df_explore.loc[:,temp_cat.index].fillna('Missing-NA')
        genMsg(['Qualitative', df_cat.shape[1]])
        temp_cat = temp_cat.join(df_cat.describe().T).fillna('')
        temp_cat['CategoriesName'] = [ list(df_cat[fea].unique()) for fea in temp_cat.index ]
        temp_cat['%Missing'] = getMissingPct(temp_cat)
        display(temp_cat)
    
    
    ## Analyzing Quantitative Features
    temp_num = temp.loc[((temp['dtypes']=='int') | (temp['dtypes']=='float')),:]
    if (len(temp_num) > 0):
        df_num = df_explore.loc[:,temp_num.index]#.fillna('Missing-NA')
        genMsg(['Quantitative', df_num.shape[1]])
        temp_num = temp_num.join(df_num.describe().T).fillna('')
        temp_num['%Missing'] = getMissingPct(temp_num)
        display(temp_num)
    # if temp_cat['dtypes'][i] == 'float', 'int', 'O'

    if len(temp)!= len(temp_tim) + len(temp_cat) + len(temp_num):
        print("Some columns data is missing b/c of data type")
    
    dit = {'TimeBased': temp_tim, 'Caterorical': temp_cat, 'Numerical': temp_num}
    return dit

In [97]:
## Reading Data
# df = pd.read_excel("Superstore.xls")

primDf = pd.read_csv('data/WC_DS_Ex1_Pri_Sales.csv')
secDf = pd.read_csv('data/WC_DS_Ex1_Sec_Sales.csv')
invDf = pd.read_csv('data/WC_DS_Ex1_Inv.csv')

df_li = {'Primary':primDf, 
         'Secondary': secDf, 
         'Inventory': invDf
        }


In [100]:
## transfering the feature "SKU_Code" to object and then viewing the result
for df_name in df_li:
    print('"{}" dataframe shape:  {}'.format(df_name, df_li[df_name].shape))
    
    ## Changing feature data type to object
    df_li[df_name][['SKU_Code']] = df_li[df_name][['SKU_Code']].astype(str)
    
    ## Changing feature data type to datetime
    # df_li[df_name]['Date'] = [ datetime.strptime(ele, '%Y-%m-%d') for ele in df_li[df_name]['Date'] ] # %H:%M:%S
    df_li[df_name]['Date'] = pd.to_datetime(df_li[df_name]['Date'],format='%Y-%m-%d') 
    
    display(df_li[df_name].head())
    _ = DatasetPrimAnalysis(df_li[df_name])
    print('****'*25,'\n\n')

"Primary" dataframe shape:  (21115, 5)


,Store_Code,SKU_Code,Category,Date,Qty
0,Store1,603132,Cat1,2015-12-11,2
1,Store1,603744,Cat1,2015-12-11,1
2,Store1,610017,Cat3,2015-12-11,1
3,Store1,610572,Cat1,2015-12-11,1
4,Store1,614427,Cat2,2015-12-11,1


Overall dataset shape : (21115, 5)
____________| Number of feature/s which are Time based : 1 |____________


,IsNullSum,dtypes,IsNaSum,count,unique,top,freq,first,last,%Missing
Date,0,datetime64[ns],0,21115,66,2017-03-31,1428,2015-12-11,2017-12-25,0.0


____________| Number of feature/s which are Qualitative : 3 |____________


...

____________| Number of feature/s which are Qualitative : 3 |____________


,IsNullSum,dtypes,IsNaSum,count,unique,top,freq,CategoriesName,%Missing
Store_Code,0,object,0,58835,3,Store1,21748,"[Store3, Store1, Store2]",0.0
SKU_Code,0,object,0,58835,2283,600366,113,"[602994, 602991, 602988, 602985, 602979, 60298...",0.0
Category,0,object,0,58835,4,Cat1,23015,"[Cat1, Cat2, Cat4, Cat3]",0.0


____________| Number of feature/s which are Quantitative : 1 |____________


,IsNullSum,dtypes,IsNaSum,count,mean,std,min,25%,50%,75%,max,%Missing
SOH,0,int64,0,58835.0,1.951355,1.227142,1.0,1.0,2.0,2.0,15.0,0.0


**************************************************************************************************** 




In [144]:
## splitting the dataset

def namestr(obj, namespace): #namestr(primDf, globals())[0]
    return [name for name in namespace if namespace[name] is obj]

def splitTimeSeriesData(df, split_date, msg =True):
    ''' Split DataFramee based on Split Date
    '''
    df_train = df.loc[df['Date'] <= split_date]
    df_test = df.loc[df['Date'] > split_date]
    if msg: print('Original DataFrame Shape: {} \n\tTrain Shape: {}\tTest Shape:{}'.format(
        df.shape, df_train.shape, df_test.shape))
    return df_train, df_test
    

split_date = pd.datetime(2017, 8, 1)
for df_name in df_li:
    '''    x='qwe'; x1= 'asd';  exec("%s = %d; %s = %d" % (x,2,x1,3)); print(qwe, asd)
    '''
    varName = namestr(df_li[df_name], globals())[0]
    exec("{0}, {1} = splitTimeSeriesData(df_li[df_name], split_date)".format(varName+'_train', varName+'_test'))

print('\nOriginal "Primary" DataFrame Shape: {} \n\tTrain Shape: {}\tTest Shape:{}'.format(
    primDf.shape, primDf_train.shape, primDf_test.shape))

Original DataFrame Shape: (21115, 5) 
	Train Shape: (16810, 5)	Test Shape:(4305, 5)
Original DataFrame Shape: (24554, 7) 
	Train Shape: (20127, 7)	Test Shape:(4427, 7)
Original DataFrame Shape: (58835, 5) 
	Train Shape: (42816, 5)	Test Shape:(16019, 5)

Original "Primary" DataFrame Shape: (21115, 5) 
	Train Shape: (16810, 5)	Test Shape:(4305, 5)


Original DataFrame Shape: (21115, 5) 
	Train Shape: (16810, 5)	Test Shape:(4305, 5)


(16810, 5)

## Answer the following

1. Aggregate the Sales_Qty for each Store-SKU at a month level; detect any Outliers in the Sales_Qty for each Store-SKU combination and apply an outlier treatment on the same.
Specify the outlier treatment technique.

2. Estimate the level of promotions (Discount%) for each Category-Store level at a month level - remove any outliers / inconsistencies from this, and specify the technique used; the level of promotions is defined as Discount% = (1 - sum of SP / sum of MRP)

3. Estimate the inventory levels at a weekly level for each Store-SKU by interpolating missing values from data on secondary and primary sales; the following equation holds true in general: (​ you can do this for a shorter period of Jan 2017 to Mar 2017 ) ​ Closing inventory on day [t] = Closing inventory on day [t-1] - Secondary (sales - returns) on day [t] + Primary (sales - returns) on day [t]
NOTE:
a. Secondary sales is the file named “WC_DS_Ex1_Sec_Sales.csv” - and it refers
to sales from stores to customers (and returns by customers)
b. Primary sales is the file name “WC_DS_Ex1_Pri_Sales.csv” - and it refers to
stock movements from retailer WH to stores (and returns back to WH)
c. Returns in both datasets are indicated by negative values in ‘Sales_Qty’ and ‘Qty’
fields respectively
4. The inventory estimations in Question 3 will have data inconsistencies - take any
assumption to resolve them and explain that assumption
5. Using the Secondary sales data and inventory series from Question 3, determine
average out-of-stock percentage (OOS%) for each Category-Store combination at a
monthly level; the OOS % is defined as:
OOS % = 1 - {Average of no. of unique SKUs in stock each day
/ No. of unique SKUs in stock over the entire month}
(for each Category-Store combination each month)
WyngCommerce
Copyright © 2018. All rights reserved | Confidential
3(​ Again - do this for a short period of Jan 2017 - Mar 2017; for forecasting you can
assume that the retailer will experience similar OOS levels in Jan-Mar 2018 ​ )
6. Using the historical secondary sales, inventory series, OOS% levels and promotion
levels, determine the demand for each Store-SKU combination at a monthly level for the
forecast period; use any forecasting technique that you’re comfortable with (you may use
multiple techniques)
7. Explain the approach for Question 6 clearly e.g. dividing data into train, validation and
test sets, choice of technique used, metric(s) used to evaluate the results
8. If any of the above steps is becoming computationally too expensive or taking too long;
you are free to either simplify them or reduce the complexity (e.g. impute weekly
inventory positions instead of daily)